# Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
import numpy
# fix random seed for reproducibility
numpy.random.seed(42)

Using TensorFlow backend.


In [2]:
df = pd.read_csv('/Users/ksmith/Documents/Code/DS1/Unit4/DS-Unit-4-Sprint-3-Neural-Networks/module4-Hyperparameter-Tuning/WA_Fn-UseC_-Telco-Customer-Churn.csv')

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.describe(include='all')

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
count,7043,7043,7043.000000,7043,7043,7043.000000,7043,7043,7043,7043,...,7043,7043,7043,7043,7043,7043,7043,7043.000000,7043,7043
unique,7043,2,NaN,2,2,NaN,2,3,3,3,...,3,3,3,3,3,2,4,NaN,6531,2
top,2799-ARNLO,Male,NaN,No,No,NaN,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,NaN,20.2,No
freq,1,3555,NaN,3641,4933,NaN,6361,3390,3096,3498,...,3095,3473,2810,2785,3875,4171,2365,NaN,11,5174
mean,NaN,NaN,0.162147,NaN,NaN,32.371149,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.761692,NaN,NaN
std,NaN,NaN,0.368612,NaN,NaN,24.559481,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.090047,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.250000,NaN,NaN
25%,NaN,NaN,0.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.500000,NaN,NaN
50%,NaN,NaN,0.000000,NaN,NaN,29.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.350000,NaN,NaN
75%,NaN,NaN,0.000000,NaN,NaN,55.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.850000,NaN,NaN


In [4]:
df.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [5]:
df = df.drop(columns = 'customerID')
df.TotalCharges = df.TotalCharges.replace(' ', 0.0)
df.TotalCharges = df.TotalCharges.astype(float)
df.tenure = df.tenure.astype(float)
df = pd.get_dummies(df)
df.TotalCharges = df.TotalCharges.replace(' ', 0.0)
df.TotalCharges = df.TotalCharges.astype(float)
df.tenure = df.tenure.astype(float)
df = pd.get_dummies(df)
df.shape

(7043, 47)

In [6]:
df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
0,0,1.0,29.85,29.85,1,0,0,1,1,0,...,0,0,0,1,0,0,1,0,1,0
1,0,34.0,56.95,1889.50,0,1,1,0,1,0,...,1,0,1,0,0,0,0,1,1,0
2,0,2.0,53.85,108.15,0,1,1,0,1,0,...,0,0,0,1,0,0,0,1,0,1
3,0,45.0,42.30,1840.75,0,1,1,0,1,0,...,1,0,1,0,1,0,0,0,1,0
4,0,2.0,70.70,151.65,1,0,1,0,1,0,...,0,0,0,1,0,0,1,0,0,1


In [7]:
from sklearn.preprocessing import StandardScaler

y = df.Churn_Yes.values
X = df.drop(columns = ['Churn_No','Churn_Yes']).values

# Scale inputs
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
# Function to create model, required for KerasClassifier
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=45, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
batch_size = [20]
epochs = [20]
param_grid = dict(batch_size=batch_size, epochs=epochs)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
print()
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
4695/4695 [==============================] - 0s 67us/step - loss: 0.5724 - acc: 0.6973
Epoch 2/20
4695/4695 [==============================] - 0s 33us/step - loss: 0.4434 - acc: 0.7923
Epoch 3/20
4695/4695 [==============================] - 0s 33us/step - loss: 0.4322 - acc: 0.7966
Epoch 4/20
4695/4695 [==============================] - 0s 34us/step - loss: 0.4269 - acc: 0.7996
Epoch 5/20
4695/4695 [==============================] - 0s 33us/step - loss: 0.4227 - acc: 0.8055
Epoch 6/20
4695/4695 [==============================] - 0s 35us/step - loss: 0.4204 - acc: 0.8051
Epoch 7/20
4695/4695 [==============================] - 0s 32us/step - loss: 0.4192 - acc: 0.8047
Epoch 8/20
4695/4695 [==============================] - 0s 32us/step - loss: 0.4175 - acc: 0.8083
Epoch 9/20
4695/4695 [==============================] - 0s 35us/step - loss: 0.4168 - acc: 0.807

7043/7043 [==============================] - 0s 34us/step - loss: 0.4038 - acc: 0.8076
Epoch 15/20
7043/7043 [==============================] - 0s 34us/step - loss: 0.4051 - acc: 0.8090
Epoch 16/20
7043/7043 [==============================] - 0s 33us/step - loss: 0.4045 - acc: 0.8089
Epoch 17/20
7043/7043 [==============================] - 0s 33us/step - loss: 0.4035 - acc: 0.8100
Epoch 18/20
7043/7043 [==============================] - 0s 35us/step - loss: 0.4029 - acc: 0.8066
Epoch 19/20
7043/7043 [==============================] - 0s 41us/step - loss: 0.4026 - acc: 0.8086
Epoch 20/20
7043/7043 [==============================] - 0s 38us/step - loss: 0.4025 - acc: 0.8112
Best: 0.800795116157808 using {'batch_size': 20, 'epochs': 20}

Means: 0.800795116157808, Stdev: 0.0066465948495364495 with: {'batch_size': 20, 'epochs': 20}


In [10]:
# Report Results
print(f"Best: {grid_result.best_score_:.3f} using {grid_result.best_params_}")
print()
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean:.3f}, Stdev: {stdev:.3f} with: {param}")

Best: 0.801 using {'batch_size': 20, 'epochs': 20}

Means: 0.801, Stdev: 0.007 with: {'batch_size': 20, 'epochs': 20}


In [11]:
# Function to create model, required for KerasClassifier
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=45, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Epoch 1/20
4695/4695 [==============================] - 1s 126us/step - loss: 0.5083 - acc: 0.7357
Epoch 2/20
4695/4695 [==============================] - 0s 68us/step - loss: 0.4428 - acc: 0.7911
Epoch 3/20
4695/4695 [==============================] - 0s 68us/step - loss: 0.4325 - acc: 0.7998
Epoch 4/20
4695/4695 [==============================] - 0s 67us/step - loss: 0.4265 - acc: 0.8011
Epoch 5/20
4695/4695 [==============================] - 0s 67us/step - loss: 0.4224 - acc: 0.8079
Epoch 6/20
4695/4695 [==============================] - 0s 67us/step - loss: 0.4200 - acc: 0.8045
Epoch 7/20
4695/4695 [==============================] - 0s 67us/step - loss: 0.4187 - acc: 0.8064
Epoch 8/20
4695/4695 [==============================] - 0s 67us/step - loss: 0.4165 - acc: 0.8085
Epoch 9/20
4695/4695 [==============================] - 0s 68us/step - loss: 0.4158 - acc: 0.8072
Epoch 10/20
4695/4695 [==============================] - 0s 67us/step - loss: 0.4149 - acc: 0.8079
Epoch 11/20
4695/4

4695/4695 [==============================] - 0s 13us/step
Epoch 1/20
4695/4695 [==============================] - 1s 111us/step - loss: 0.4991 - acc: 0.7478
Epoch 2/20
4695/4695 [==============================] - 0s 35us/step - loss: 0.4386 - acc: 0.7940
Epoch 3/20
4695/4695 [==============================] - 0s 37us/step - loss: 0.4276 - acc: 0.8004
Epoch 4/20
4695/4695 [==============================] - 0s 35us/step - loss: 0.4208 - acc: 0.7985
Epoch 5/20
4695/4695 [==============================] - 0s 35us/step - loss: 0.4169 - acc: 0.8028
Epoch 6/20
4695/4695 [==============================] - 0s 36us/step - loss: 0.4135 - acc: 0.8049
Epoch 7/20
4695/4695 [==============================] - 0s 36us/step - loss: 0.4108 - acc: 0.8055
Epoch 8/20
4695/4695 [==============================] - 0s 35us/step - loss: 0.4085 - acc: 0.8068
Epoch 9/20
4695/4695 [==============================] - 0s 36us/step - loss: 0.4074 - acc: 0.8064
Epoch 10/20
4695/4695 [==============================] - 0s

4695/4695 [==============================] - 0s 19us/step - loss: 0.4005 - acc: 0.8126
Epoch 19/20
4695/4695 [==============================] - 0s 19us/step - loss: 0.4002 - acc: 0.8102
Epoch 20/20
4695/4695 [==============================] - 0s 7us/step
Epoch 1/20
4696/4696 [==============================] - 1s 118us/step - loss: 0.5722 - acc: 0.6846
Epoch 2/20
4696/4696 [==============================] - 0s 19us/step - loss: 0.4424 - acc: 0.7830
Epoch 3/20
4696/4696 [==============================] - 0s 18us/step - loss: 0.4270 - acc: 0.7909
Epoch 4/20
4696/4696 [==============================] - 0s 18us/step - loss: 0.4198 - acc: 0.7958
Epoch 5/20
4696/4696 [==============================] - 0s 18us/step - loss: 0.4150 - acc: 0.7981
Epoch 6/20
4696/4696 [==============================] - 0s 18us/step - loss: 0.4111 - acc: 0.8020
Epoch 7/20
4696/4696 [==============================] - 0s 19us/step - loss: 0.4089 - acc: 0.8034
Epoch 8/20
4696/4696 [==============================] - 0s

4696/4696 [==============================] - 0s 13us/step - loss: 0.4014 - acc: 0.8096
Epoch 17/20
4696/4696 [==============================] - 0s 13us/step - loss: 0.4012 - acc: 0.8107
Epoch 18/20
4696/4696 [==============================] - 0s 13us/step - loss: 0.4003 - acc: 0.8107
Epoch 19/20
4696/4696 [==============================] - 0s 13us/step - loss: 0.3994 - acc: 0.8122
Epoch 20/20
4696/4696 [==============================] - 0s 5us/step
Epoch 1/20
4695/4695 [==============================] - 1s 133us/step - loss: 0.7256 - acc: 0.5565
Epoch 2/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.5332 - acc: 0.7210
Epoch 3/20
4695/4695 [==============================] - 0s 10us/step - loss: 0.4735 - acc: 0.7657
Epoch 4/20
4695/4695 [==============================] - 0s 11us/step - loss: 0.4516 - acc: 0.7851
Epoch 5/20
4695/4695 [==============================] - 0s 12us/step - loss: 0.4409 - acc: 0.7921
Epoch 6/20
4695/4695 [==============================] - 0

4695/4695 [==============================] - 0s 8us/step - loss: 0.4237 - acc: 0.8011
Epoch 15/20
4695/4695 [==============================] - 0s 8us/step - loss: 0.4226 - acc: 0.8028
Epoch 16/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4215 - acc: 0.8023
Epoch 17/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4206 - acc: 0.8049
Epoch 18/20
4695/4695 [==============================] - 0s 10us/step - loss: 0.4194 - acc: 0.8045
Epoch 19/20
4695/4695 [==============================] - 0s 10us/step - loss: 0.4192 - acc: 0.8051
Epoch 20/20
4695/4695 [==============================] - 0s 4us/step
Epoch 1/20
4695/4695 [==============================] - 1s 154us/step - loss: 0.6782 - acc: 0.6011
Epoch 2/20
4695/4695 [==============================] - 0s 8us/step - loss: 0.5332 - acc: 0.7448
Epoch 3/20
4695/4695 [==============================] - 0s 8us/step - loss: 0.4785 - acc: 0.7744
Epoch 4/20
4695/4695 [==============================] - 0s 9

In [12]:
# define the grid search parameters
param_grid = {'batch_size': [80,90,100],
              'epochs': [10, 20, 40, 60]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/10
4695/4695 [==============================] - 1s 176us/step - loss: 0.5470 - acc: 0.7086
Epoch 2/10
4695/4695 [==============================] - 0s 12us/step - loss: 0.4736 - acc: 0.7710
Epoch 3/10
4695/4695 [==============================] - 0s 12us/step - loss: 0.4521 - acc: 0.7859
Epoch 4/10
4695/4695 [==============================] - 0s 13us/step - loss: 0.4412 - acc: 0.7911
Epoch 5/10
4695/4695 [==============================] - 0s 14us/step - loss: 0.4350 - acc: 0.7974
Epoch 6/10
4695/4695 [==============================] - 0s 12us/step - loss: 0.4303 - acc: 0.7996
Epoch 7/10
4695/4695 [==============================] - 0s 13us/step - loss: 0.4274 - acc: 0.8028
Epoch 8/10
4695/4695 [==============================] - 0s 12us/step - loss: 0.4256 - acc: 0.8030
Epoch 9/10
4695/4695 [==============================] - 0s 11us/step - loss: 0.4230 - acc: 0.8045
Epoch 10/10
4695/4695 [==============================] - 0s 6us/step
Epoch 1/10
4695/4695 [==========================

4696/4696 [==============================] - 0s 12us/step - loss: 0.4112 - acc: 0.8022
Epoch 9/20
4696/4696 [==============================] - 0s 12us/step - loss: 0.4095 - acc: 0.8069
Epoch 10/20
4696/4696 [==============================] - 0s 12us/step - loss: 0.4082 - acc: 0.8058
Epoch 11/20
4696/4696 [==============================] - 0s 12us/step - loss: 0.4067 - acc: 0.8077
Epoch 12/20
4696/4696 [==============================] - 0s 12us/step - loss: 0.4059 - acc: 0.8064
Epoch 13/20
4696/4696 [==============================] - 0s 12us/step - loss: 0.4045 - acc: 0.8101
Epoch 14/20
4696/4696 [==============================] - 0s 11us/step - loss: 0.4039 - acc: 0.8098
Epoch 15/20
4696/4696 [==============================] - 0s 11us/step - loss: 0.4029 - acc: 0.8096
Epoch 16/20
4696/4696 [==============================] - 0s 11us/step - loss: 0.4017 - acc: 0.8107
Epoch 17/20
4696/4696 [==============================] - 0s 11us/step - loss: 0.4010 - acc: 0.8118
Epoch 18/20
4696/4696 [

4695/4695 [==============================] - 0s 11us/step - loss: 0.4016 - acc: 0.8113
Epoch 29/40
4695/4695 [==============================] - 0s 12us/step - loss: 0.4011 - acc: 0.8109
Epoch 30/40
4695/4695 [==============================] - 0s 13us/step - loss: 0.4004 - acc: 0.8136
Epoch 31/40
4695/4695 [==============================] - 0s 19us/step - loss: 0.3994 - acc: 0.8119
Epoch 32/40
4695/4695 [==============================] - 0s 15us/step - loss: 0.3998 - acc: 0.8134
Epoch 33/40
4695/4695 [==============================] - 0s 15us/step - loss: 0.3983 - acc: 0.8149
Epoch 34/40
4695/4695 [==============================] - 0s 12us/step - loss: 0.3978 - acc: 0.8128
Epoch 35/40
4695/4695 [==============================] - 0s 14us/step - loss: 0.3979 - acc: 0.8126
Epoch 36/40
4695/4695 [==============================] - 0s 12us/step - loss: 0.3970 - acc: 0.8136
Epoch 37/40
4695/4695 [==============================] - 0s 13us/step - loss: 0.3965 - acc: 0.8109
Epoch 38/40
4695/4695 

4695/4695 [==============================] - 0s 12us/step - loss: 0.4098 - acc: 0.8119
Epoch 29/60
4695/4695 [==============================] - 0s 12us/step - loss: 0.4091 - acc: 0.8119
Epoch 30/60
4695/4695 [==============================] - 0s 13us/step - loss: 0.4088 - acc: 0.8111
Epoch 31/60
4695/4695 [==============================] - 0s 12us/step - loss: 0.4084 - acc: 0.8128
Epoch 32/60
4695/4695 [==============================] - 0s 13us/step - loss: 0.4084 - acc: 0.8130
Epoch 33/60
4695/4695 [==============================] - 0s 12us/step - loss: 0.4079 - acc: 0.8121
Epoch 34/60
4695/4695 [==============================] - 0s 13us/step - loss: 0.4076 - acc: 0.8121
Epoch 35/60
4695/4695 [==============================] - 0s 11us/step - loss: 0.4076 - acc: 0.8138
Epoch 36/60
4695/4695 [==============================] - 0s 13us/step - loss: 0.4074 - acc: 0.8124
Epoch 37/60
4695/4695 [==============================] - 0s 13us/step - loss: 0.4072 - acc: 0.8121
Epoch 38/60
4695/4695 

4695/4695 [==============================] - 0s 15us/step - loss: 0.3932 - acc: 0.8160
Epoch 50/60
4695/4695 [==============================] - 0s 16us/step - loss: 0.3924 - acc: 0.8173
Epoch 51/60
4695/4695 [==============================] - 0s 16us/step - loss: 0.3924 - acc: 0.8177
Epoch 52/60
4695/4695 [==============================] - 0s 15us/step - loss: 0.3917 - acc: 0.8143
Epoch 53/60
4695/4695 [==============================] - 0s 15us/step - loss: 0.3921 - acc: 0.8187
Epoch 54/60
4695/4695 [==============================] - 0s 14us/step - loss: 0.3911 - acc: 0.8166
Epoch 55/60
4695/4695 [==============================] - 0s 14us/step - loss: 0.3912 - acc: 0.8147
Epoch 56/60
4695/4695 [==============================] - 0s 15us/step - loss: 0.3914 - acc: 0.8170
Epoch 57/60
4695/4695 [==============================] - 0s 16us/step - loss: 0.3903 - acc: 0.8160
Epoch 58/60
4695/4695 [==============================] - 0s 15us/step - loss: 0.3911 - acc: 0.8143
Epoch 59/60
4695/4695 

...

4695/4695 [==============================] - 0s 13us/step - loss: 0.4328 - acc: 0.7934
Epoch 7/10
4695/4695 [==============================] - 0s 13us/step - loss: 0.4287 - acc: 0.7947
Epoch 8/10
4695/4695 [==============================] - 0s 13us/step - loss: 0.4254 - acc: 0.7981
Epoch 9/10
4695/4695 [==============================] - 0s 13us/step - loss: 0.4227 - acc: 0.7972
Epoch 10/10
4695/4695 [==============================] - 0s 8us/step
Epoch 1/10
4696/4696 [==============================] - 2s 326us/step - loss: 0.6065 - acc: 0.6540
Epoch 2/10
4696/4696 [==============================] - 0s 14us/step - loss: 0.5008 - acc: 0.7566
Epoch 3/10
4696/4696 [==============================] - 0s 13us/step - loss: 0.4593 - acc: 0.7773
Epoch 4/10
4696/4696 [==============================] - 0s 14us/step - loss: 0.4411 - acc: 0.7871
Epoch 5/10
4696/4696 [==============================] - 0s 13us/step - loss: 0.4311 - acc: 0.7930
Epoch 6/10
4696/4696 [==============================] - 0s 

4695/4695 [==============================] - 0s 13us/step - loss: 0.4671 - acc: 0.7644
Epoch 4/40
4695/4695 [==============================] - 0s 14us/step - loss: 0.4534 - acc: 0.7734
Epoch 5/40
4695/4695 [==============================] - 0s 14us/step - loss: 0.4452 - acc: 0.7821
Epoch 6/40
4695/4695 [==============================] - 0s 14us/step - loss: 0.4395 - acc: 0.7838
Epoch 7/40
4695/4695 [==============================] - 0s 13us/step - loss: 0.4346 - acc: 0.7866
Epoch 8/40
4695/4695 [==============================] - 0s 14us/step - loss: 0.4313 - acc: 0.7906
Epoch 9/40
4695/4695 [==============================] - 0s 16us/step - loss: 0.4281 - acc: 0.7928
Epoch 10/40
4695/4695 [==============================] - 0s 16us/step - loss: 0.4257 - acc: 0.7930
Epoch 11/40
4695/4695 [==============================] - 0s 14us/step - loss: 0.4236 - acc: 0.7960
Epoch 12/40
4695/4695 [==============================] - 0s 12us/step - loss: 0.4215 - acc: 0.7994
Epoch 13/40
4695/4695 [=====

4696/4696 [==============================] - 0s 14us/step - loss: 0.4861 - acc: 0.7615
Epoch 4/40
4696/4696 [==============================] - 0s 16us/step - loss: 0.4593 - acc: 0.7770
Epoch 5/40
4696/4696 [==============================] - 0s 15us/step - loss: 0.4451 - acc: 0.7849
Epoch 6/40
4696/4696 [==============================] - 0s 14us/step - loss: 0.4356 - acc: 0.7941
Epoch 7/40
4696/4696 [==============================] - 0s 14us/step - loss: 0.4287 - acc: 0.7990
Epoch 8/40
4696/4696 [==============================] - 0s 14us/step - loss: 0.4233 - acc: 0.7998
Epoch 9/40
4696/4696 [==============================] - 0s 14us/step - loss: 0.4194 - acc: 0.7986
Epoch 10/40
4696/4696 [==============================] - 0s 15us/step - loss: 0.4163 - acc: 0.8024
Epoch 11/40
4696/4696 [==============================] - 0s 15us/step - loss: 0.4137 - acc: 0.8049
Epoch 12/40
4696/4696 [==============================] - 0s 16us/step - loss: 0.4119 - acc: 0.8052
Epoch 13/40
4696/4696 [=====

4695/4695 [==============================] - 0s 16us/step - loss: 0.4081 - acc: 0.8102
Epoch 45/60
4695/4695 [==============================] - 0s 16us/step - loss: 0.4083 - acc: 0.8119
Epoch 46/60
4695/4695 [==============================] - 0s 15us/step - loss: 0.4081 - acc: 0.8113
Epoch 47/60
4695/4695 [==============================] - 0s 15us/step - loss: 0.4075 - acc: 0.8115
Epoch 48/60
4695/4695 [==============================] - 0s 15us/step - loss: 0.4076 - acc: 0.8130
Epoch 49/60
4695/4695 [==============================] - 0s 15us/step - loss: 0.4071 - acc: 0.8113
Epoch 50/60
4695/4695 [==============================] - 0s 14us/step - loss: 0.4069 - acc: 0.8121
Epoch 51/60
4695/4695 [==============================] - 0s 15us/step - loss: 0.4067 - acc: 0.8136
Epoch 52/60
4695/4695 [==============================] - 0s 15us/step - loss: 0.4064 - acc: 0.8147
Epoch 53/60
4695/4695 [==============================] - 0s 16us/step - loss: 0.4071 - acc: 0.8121
Epoch 54/60
4695/4695 

4696/4696 [==============================] - 0s 16us/step - loss: 0.4782 - acc: 0.7807
Epoch 5/60
4696/4696 [==============================] - 0s 15us/step - loss: 0.4526 - acc: 0.7858
Epoch 6/60
4696/4696 [==============================] - 0s 15us/step - loss: 0.4379 - acc: 0.7883
Epoch 7/60
4696/4696 [==============================] - 0s 16us/step - loss: 0.4287 - acc: 0.7949
Epoch 8/60
4696/4696 [==============================] - 0s 15us/step - loss: 0.4229 - acc: 0.7973
Epoch 9/60
4696/4696 [==============================] - 0s 15us/step - loss: 0.4192 - acc: 0.7977
Epoch 10/60
4696/4696 [==============================] - 0s 15us/step - loss: 0.4163 - acc: 0.8011
Epoch 11/60
4696/4696 [==============================] - 0s 15us/step - loss: 0.4142 - acc: 0.8020
Epoch 12/60
4696/4696 [==============================] - 0s 15us/step - loss: 0.4125 - acc: 0.8043
Epoch 13/60
4696/4696 [==============================] - 0s 15us/step - loss: 0.4112 - acc: 0.8054
Epoch 14/60
4696/4696 [====

In [8]:
from keras.optimizers import Adam

# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=45, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	optimizer = Adam(lr=learn_rate)
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=80, verbose=1)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, .2]
param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)
# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
4695/4695 [==============================] - 0s 44us/step - loss: 0.7130 - acc: 0.5974
Epoch 2/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.5224 - acc: 0.7401
Epoch 3/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4643 - acc: 0.7810
Epoch 4/20
4695/4695 [==============================] - 0s 10us/step - loss: 0.4442 - acc: 0.7938
Epoch 5/20
4695/4695 [==============================] - 0s 10us/step - loss: 0.4361 - acc: 0.7974
Epoch 6/20
4695/4695 [==============================] - 0s 10us/step - loss: 0.4314 - acc: 0.8004
Epoch 7/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4285 - acc: 0.8000
Epoch 8/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4256 - acc: 0.8011
Epoch 9/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4238 - acc: 0.8034
Epo

4695/4695 [==============================] - 0s 9us/step - loss: 0.4114 - acc: 0.8092
Epoch 16/20
4695/4695 [==============================] - 0s 10us/step - loss: 0.4077 - acc: 0.8119
Epoch 17/20
4695/4695 [==============================] - 0s 10us/step - loss: 0.4057 - acc: 0.8124
Epoch 18/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4047 - acc: 0.8128
Epoch 19/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4070 - acc: 0.8124
Epoch 20/20
4695/4695 [==============================] - 0s 4us/step
Epoch 1/20
4695/4695 [==============================] - 0s 62us/step - loss: 0.4794 - acc: 0.7653
Epoch 2/20
4695/4695 [==============================] - 0s 8us/step - loss: 0.4215 - acc: 0.7983
Epoch 3/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4141 - acc: 0.8019
Epoch 4/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4098 - acc: 0.8017
Epoch 5/20
4695/4695 [==============================] - 0s 9us

4695/4695 [==============================] - 0s 9us/step - loss: 0.4231 - acc: 0.8011
Epoch 15/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4302 - acc: 0.7915
Epoch 16/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4259 - acc: 0.7991
Epoch 17/20
4695/4695 [==============================] - 0s 10us/step - loss: 0.4225 - acc: 0.7979
Epoch 18/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4259 - acc: 0.8013
Epoch 19/20
4695/4695 [==============================] - 0s 9us/step - loss: 0.4284 - acc: 0.7979
Epoch 20/20
4695/4695 [==============================] - 0s 4us/step
Epoch 1/20
4696/4696 [==============================] - 0s 84us/step - loss: 0.4826 - acc: 0.7615
Epoch 2/20
4696/4696 [==============================] - 0s 9us/step - loss: 0.4498 - acc: 0.7796
Epoch 3/20
4696/4696 [==============================] - 0s 9us/step - loss: 0.4384 - acc: 0.7902
Epoch 4/20
4696/4696 [==============================] - 0s 9us

Epoch 13/20
4696/4696 [==============================] - 0s 10us/step - loss: 0.4996 - acc: 0.7236
Epoch 14/20
4696/4696 [==============================] - 0s 10us/step - loss: 0.4893 - acc: 0.7202
Epoch 15/20
4696/4696 [==============================] - 0s 9us/step - loss: 0.4825 - acc: 0.7368
Epoch 16/20
4696/4696 [==============================] - 0s 10us/step - loss: 0.4994 - acc: 0.7306
Epoch 17/20
4696/4696 [==============================] - 0s 9us/step - loss: 0.4944 - acc: 0.7330
Epoch 18/20
4696/4696 [==============================] - 0s 10us/step - loss: 0.4891 - acc: 0.7368
Epoch 19/20
4696/4696 [==============================] - 0s 9us/step - loss: 0.4874 - acc: 0.7178
Epoch 20/20
4696/4696 [==============================] - 0s 4us/step
Epoch 1/20
7043/7043 [==============================] - 1s 74us/step - loss: 0.7693 - acc: 0.5554
Epoch 2/20
7043/7043 [==============================] - 0s 10us/step - loss: 0.4998 - acc: 0.7412
Epoch 3/20
7043/7043 [=======================

In [10]:
from keras.optimizers import SGD

# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
  # create model
  model = Sequential()
  model.add(Dense(12, input_dim=45, activation='sigmoid'))
  model.add(Dense(1, activation='sigmoid'))
  optimizer = SGD(lr=learn_rate, momentum=momentum)
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=80, verbose=1)

# Create param grid
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Epoch 1/20
4695/4695 [==============================] - 0s 104us/step - loss: 0.7706 - acc: 0.4089
Epoch 2/20
4695/4695 [==============================] - 0s 12us/step - loss: 0.7444 - acc: 0.4373
Epoch 3/20
4695/4695 [==============================] - 0s 11us/step - loss: 0.7215 - acc: 0.4705
Epoch 4/20
4695/4695 [==============================] - 0s 11us/step - loss: 0.7018 - acc: 0.4939
Epoch 5/20
4695/4695 [==============================] - 0s 11us/step - loss: 0.6844 - acc: 0.5269
Epoch 6/20
4695/4695 [==============================] - 0s 12us/step - loss: 0.6693 - acc: 0.5589
Epoch 7/20
4695/4695 [==============================] - 0s 11us/step - loss: 0.6560 - acc: 0.5951
Epoch 8/20
4695/4695 [==============================] - 0s 12us/step - loss: 0.6445 - acc: 0.6217
Epoch 9/20
4695/4695 [==============================] - 0s 12us/step - loss: 0.6341 - acc: 0.6552
Epoch 10/20
4695/4695 [==============================] - 0s 12us/step - loss: 0.6249 - acc: 0.6799
Epoch 11/20
4695/4

4695/4695 [==============================] - 0s 6us/step
Epoch 1/20
4695/4695 [==============================] - 1s 145us/step - loss: 0.8646 - acc: 0.2901
Epoch 2/20
4695/4695 [==============================] - 0s 15us/step - loss: 0.8205 - acc: 0.3276
Epoch 3/20
4695/4695 [==============================] - 0s 11us/step - loss: 0.7833 - acc: 0.3842
Epoch 4/20
4695/4695 [==============================] - 0s 12us/step - loss: 0.7520 - acc: 0.4324
Epoch 5/20
4695/4695 [==============================] - 0s 12us/step - loss: 0.7256 - acc: 0.4716
Epoch 6/20
4695/4695 [==============================] - 0s 11us/step - loss: 0.7034 - acc: 0.5084
Epoch 7/20
4695/4695 [==============================] - 0s 11us/step - loss: 0.6845 - acc: 0.5450
Epoch 8/20
4695/4695 [==============================] - 0s 11us/step - loss: 0.6684 - acc: 0.5810
Epoch 9/20
4695/4695 [==============================] - 0s 12us/step - loss: 0.6546 - acc: 0.6109
Epoch 10/20
4695/4695 [==============================] - 0s 

4695/4695 [==============================] - 0s 16us/step - loss: 0.5747 - acc: 0.7327
Epoch 19/20
4695/4695 [==============================] - 0s 11us/step - loss: 0.5724 - acc: 0.7327
Epoch 20/20
4695/4695 [==============================] - 0s 5us/step
Epoch 1/20
4696/4696 [==============================] - 1s 142us/step - loss: 0.7108 - acc: 0.5106
Epoch 2/20
4696/4696 [==============================] - 0s 11us/step - loss: 0.6896 - acc: 0.5583
Epoch 3/20
4696/4696 [==============================] - 0s 11us/step - loss: 0.6722 - acc: 0.5990
Epoch 4/20
4696/4696 [==============================] - 0s 11us/step - loss: 0.6577 - acc: 0.6288
Epoch 5/20
4696/4696 [==============================] - 0s 11us/step - loss: 0.6456 - acc: 0.6561
Epoch 6/20
4696/4696 [==============================] - 0s 11us/step - loss: 0.6352 - acc: 0.6799
Epoch 7/20
4696/4696 [==============================] - 0s 11us/step - loss: 0.6262 - acc: 0.6948
Epoch 8/20
4696/4696 [==============================] - 0s

4696/4696 [==============================] - 0s 11us/step - loss: 0.5464 - acc: 0.7379
Epoch 17/20
4696/4696 [==============================] - 0s 11us/step - loss: 0.5431 - acc: 0.7379
Epoch 18/20
4696/4696 [==============================] - 0s 11us/step - loss: 0.5399 - acc: 0.7379
Epoch 19/20
4696/4696 [==============================] - 0s 11us/step - loss: 0.5369 - acc: 0.7379
Epoch 20/20
4696/4696 [==============================] - 0s 6us/step
Epoch 1/20
4695/4695 [==============================] - 1s 187us/step - loss: 0.6020 - acc: 0.7214
Epoch 2/20
4695/4695 [==============================] - 0s 19us/step - loss: 0.5866 - acc: 0.7278
Epoch 3/20
4695/4695 [==============================] - 0s 17us/step - loss: 0.5741 - acc: 0.7308
Epoch 4/20
4695/4695 [==============================] - 0s 14us/step - loss: 0.5633 - acc: 0.7321
Epoch 5/20
4695/4695 [==============================] - 0s 12us/step - loss: 0.5539 - acc: 0.7325
Epoch 6/20
4695/4695 [==============================] - 

4695/4695 [==============================] - 0s 12us/step - loss: 0.4671 - acc: 0.7732
Epoch 15/20
4695/4695 [==============================] - 0s 14us/step - loss: 0.4644 - acc: 0.7759
Epoch 16/20
4695/4695 [==============================] - 0s 12us/step - loss: 0.4620 - acc: 0.7793
Epoch 17/20
4695/4695 [==============================] - 0s 13us/step - loss: 0.4598 - acc: 0.7783
Epoch 18/20
4695/4695 [==============================] - 0s 13us/step - loss: 0.4578 - acc: 0.7815
Epoch 19/20
4695/4695 [==============================] - 0s 12us/step - loss: 0.4561 - acc: 0.7817
Epoch 20/20
4695/4695 [==============================] - 0s 8us/step
Epoch 1/20
4695/4695 [==============================] - 1s 180us/step - loss: 0.5795 - acc: 0.7327
Epoch 2/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.5589 - acc: 0.7327
Epoch 3/20
4695/4695 [==============================] - 0s 14us/step - loss: 0.5410 - acc: 0.7327
Epoch 4/20
4695/4695 [==============================] 

...

4696/4696 [==============================] - 0s 15us/step - loss: 0.4125 - acc: 0.7992
Epoch 7/20
4696/4696 [==============================] - 0s 15us/step - loss: 0.4120 - acc: 0.8037
Epoch 8/20
4696/4696 [==============================] - 0s 15us/step - loss: 0.4110 - acc: 0.8024
Epoch 9/20
4696/4696 [==============================] - 0s 15us/step - loss: 0.4111 - acc: 0.7988
Epoch 10/20
4696/4696 [==============================] - 0s 15us/step - loss: 0.4103 - acc: 0.8022
Epoch 11/20
4696/4696 [==============================] - 0s 15us/step - loss: 0.4102 - acc: 0.8013
Epoch 12/20
4696/4696 [==============================] - 0s 15us/step - loss: 0.4086 - acc: 0.8020
Epoch 13/20
4696/4696 [==============================] - 0s 15us/step - loss: 0.4093 - acc: 0.8026
Epoch 14/20
4696/4696 [==============================] - 0s 15us/step - loss: 0.4089 - acc: 0.8028
Epoch 15/20
4696/4696 [==============================] - 0s 15us/step - loss: 0.4083 - acc: 0.8077
Epoch 16/20
4696/4696 [==

4695/4695 [==============================] - 0s 15us/step - loss: 0.4287 - acc: 0.7983
Epoch 5/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.4269 - acc: 0.8019
Epoch 6/20
4695/4695 [==============================] - 0s 24us/step - loss: 0.4267 - acc: 0.7998
Epoch 7/20
4695/4695 [==============================] - 0s 22us/step - loss: 0.4260 - acc: 0.7994
Epoch 8/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.4243 - acc: 0.8004
Epoch 9/20
4695/4695 [==============================] - 0s 15us/step - loss: 0.4235 - acc: 0.8000
Epoch 10/20
4695/4695 [==============================] - 0s 15us/step - loss: 0.4236 - acc: 0.8036
Epoch 11/20
4695/4695 [==============================] - 0s 15us/step - loss: 0.4237 - acc: 0.8032
Epoch 12/20
4695/4695 [==============================] - 0s 15us/step - loss: 0.4219 - acc: 0.8036
Epoch 13/20
4695/4695 [==============================] - 0s 15us/step - loss: 0.4221 - acc: 0.8034
Epoch 14/20
4695/4695 [====

4695/4695 [==============================] - 0s 18us/step - loss: 0.4245 - acc: 0.7953
Epoch 3/20
4695/4695 [==============================] - 0s 15us/step - loss: 0.4230 - acc: 0.7996
Epoch 4/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.4205 - acc: 0.7964
Epoch 5/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.4181 - acc: 0.8009
Epoch 6/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.4188 - acc: 0.7966
Epoch 7/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.4154 - acc: 0.8009
Epoch 8/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.4142 - acc: 0.8049
Epoch 9/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.4122 - acc: 0.8023
Epoch 10/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.4130 - acc: 0.8026
Epoch 11/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.4122 - acc: 0.8036
Epoch 12/20
4695/4695 [======

4695/4695 [==============================] - 0s 24us/step
Epoch 1/20
4696/4696 [==============================] - 3s 585us/step - loss: 0.4453 - acc: 0.7758
Epoch 2/20
4696/4696 [==============================] - 0s 29us/step - loss: 0.4200 - acc: 0.7983
Epoch 3/20
4696/4696 [==============================] - 0s 28us/step - loss: 0.4170 - acc: 0.7973
Epoch 4/20
4696/4696 [==============================] - 0s 26us/step - loss: 0.4151 - acc: 0.8000
Epoch 5/20
4696/4696 [==============================] - 0s 21us/step - loss: 0.4128 - acc: 0.8022
Epoch 6/20
4696/4696 [==============================] - 0s 20us/step - loss: 0.4105 - acc: 0.8071
Epoch 7/20
4696/4696 [==============================] - 0s 24us/step - loss: 0.4119 - acc: 0.8032
Epoch 8/20
4696/4696 [==============================] - 0s 29us/step - loss: 0.4093 - acc: 0.8030
Epoch 9/20
4696/4696 [==============================] - 0s 23us/step - loss: 0.4083 - acc: 0.8083
Epoch 10/20
4696/4696 [==============================] - 0s

Epoch 20/20
4696/4696 [==============================] - 0s 15us/step
Epoch 1/20
7043/7043 [==============================] - 3s 362us/step - loss: 0.5014 - acc: 0.7423
Epoch 2/20
7043/7043 [==============================] - 0s 21us/step - loss: 0.4375 - acc: 0.7910
Epoch 3/20
7043/7043 [==============================] - 0s 19us/step - loss: 0.4278 - acc: 0.7968
Epoch 4/20
7043/7043 [==============================] - 0s 25us/step - loss: 0.4240 - acc: 0.7992
Epoch 5/20
7043/7043 [==============================] - 0s 22us/step - loss: 0.4222 - acc: 0.7992
Epoch 6/20
7043/7043 [==============================] - 0s 22us/step - loss: 0.4215 - acc: 0.8012
Epoch 7/20
7043/7043 [==============================] - 0s 23us/step - loss: 0.4204 - acc: 0.8007
Epoch 8/20
7043/7043 [==============================] - 0s 24us/step - loss: 0.4200 - acc: 0.8026
Epoch 9/20
7043/7043 [==============================] - 0s 20us/step - loss: 0.4192 - acc: 0.8046
Epoch 10/20
7043/7043 [========================

In [11]:
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)

# Function to create model, required for KerasClassifier
def create_model(activation='relu'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=45, activation=activation))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# create model
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=80, verbose=1)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/20
4695/4695 [==============================] - 3s 553us/step - loss: 0.6633 - acc: 0.6594
Epoch 2/20
4695/4695 [==============================] - 0s 20us/step - loss: 0.5985 - acc: 0.7778
Epoch 3/20
4695/4695 [==============================] - 0s 22us/step - loss: 0.5650 - acc: 0.7817
Epoch 4/20
4695/4695 [==============================] - 0s 20us/step - loss: 0.5435 - acc: 0.7868
Epoch 5/20
4695/4695 [==============================] - 0s 25us/step - loss: 0.5273 - acc: 0.7911
Epoch 6/20
4695/4695 [==============================] - 0s 21us/step - loss: 0.5144 - acc: 0.7923
Epoch 7/20
4695/4695 [==============================] - 0s 20us/step - loss: 0.5037 - acc: 0.7966
Epoch 8/20
4695/4695 [==============================] - 0s 24us/step - loss: 0.4949 - acc: 0.7970
Epoch 9/20
4695/4695 [==============================] - 0s 21us/step - loss: 0.4874 - acc: 0.7998
Epoch 10/20
4695/4695 [==============================] - 0s 19us/step - loss: 0.4812 - acc: 0.7996
Epoch 11/20
4695/4

4695/4695 [==============================] - 0s 11us/step
Epoch 1/20
4695/4695 [==============================] - 3s 566us/step - loss: 0.5548 - acc: 0.7188
Epoch 2/20
4695/4695 [==============================] - 0s 19us/step - loss: 0.4699 - acc: 0.7734
Epoch 3/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.4410 - acc: 0.7908
Epoch 4/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.4293 - acc: 0.7964
Epoch 5/20
4695/4695 [==============================] - 0s 17us/step - loss: 0.4240 - acc: 0.8028
Epoch 6/20
4695/4695 [==============================] - 0s 20us/step - loss: 0.4200 - acc: 0.8028
Epoch 7/20
4695/4695 [==============================] - 0s 19us/step - loss: 0.4182 - acc: 0.8015
Epoch 8/20
4695/4695 [==============================] - 0s 18us/step - loss: 0.4164 - acc: 0.8026
Epoch 9/20
4695/4695 [==============================] - 0s 17us/step - loss: 0.4158 - acc: 0.8034
Epoch 10/20
4695/4695 [==============================] - 0s

4695/4695 [==============================] - 0s 30us/step - loss: 0.4071 - acc: 0.8087
Epoch 19/20
4695/4695 [==============================] - 0s 37us/step - loss: 0.4066 - acc: 0.8102
Epoch 20/20
4695/4695 [==============================] - 0s 12us/step
Epoch 1/20
4696/4696 [==============================] - 3s 598us/step - loss: 0.6502 - acc: 0.6301
Epoch 2/20
4696/4696 [==============================] - 0s 20us/step - loss: 0.5506 - acc: 0.7300
Epoch 3/20
4696/4696 [==============================] - 0s 19us/step - loss: 0.5083 - acc: 0.7594
Epoch 4/20
4696/4696 [==============================] - 0s 21us/step - loss: 0.4786 - acc: 0.7775
Epoch 5/20
4696/4696 [==============================] - 0s 19us/step - loss: 0.4569 - acc: 0.7841
Epoch 6/20
4696/4696 [==============================] - 0s 17us/step - loss: 0.4412 - acc: 0.7926
Epoch 7/20
4696/4696 [==============================] - 0s 19us/step - loss: 0.4303 - acc: 0.7949
Epoch 8/20
4696/4696 [==============================] - 0

4696/4696 [==============================] - 0s 17us/step - loss: 0.4008 - acc: 0.8143
Epoch 17/20
4696/4696 [==============================] - 0s 20us/step - loss: 0.4004 - acc: 0.8156
Epoch 18/20
4696/4696 [==============================] - 0s 17us/step - loss: 0.3996 - acc: 0.8154
Epoch 19/20
4696/4696 [==============================] - 0s 20us/step - loss: 0.3990 - acc: 0.8145
Epoch 20/20
4696/4696 [==============================] - 0s 13us/step
Epoch 1/20
4695/4695 [==============================] - 3s 605us/step - loss: 0.6716 - acc: 0.6092
Epoch 2/20
4695/4695 [==============================] - 0s 16us/step - loss: 0.5507 - acc: 0.7220
Epoch 3/20
4695/4695 [==============================] - 0s 18us/step - loss: 0.4966 - acc: 0.7602
Epoch 4/20
4695/4695 [==============================] - 0s 20us/step - loss: 0.4670 - acc: 0.7804
Epoch 5/20
4695/4695 [==============================] - 0s 17us/step - loss: 0.4505 - acc: 0.7894
Epoch 6/20
4695/4695 [==============================] -

4695/4695 [==============================] - 0s 21us/step - loss: 0.4295 - acc: 0.8028
Epoch 15/20
4695/4695 [==============================] - 0s 22us/step - loss: 0.4288 - acc: 0.8051
Epoch 16/20
4695/4695 [==============================] - 0s 18us/step - loss: 0.4278 - acc: 0.8047
Epoch 17/20
4695/4695 [==============================] - 0s 23us/step - loss: 0.4271 - acc: 0.8047
Epoch 18/20
4695/4695 [==============================] - 0s 19us/step - loss: 0.4262 - acc: 0.8049
Epoch 19/20
4695/4695 [==============================] - 0s 22us/step - loss: 0.4257 - acc: 0.8051
Epoch 20/20
4695/4695 [==============================] - 0s 14us/step
Epoch 1/20
4695/4695 [==============================] - 3s 651us/step - loss: 0.5690 - acc: 0.7327
Epoch 2/20
4695/4695 [==============================] - 0s 20us/step - loss: 0.4942 - acc: 0.7416
Epoch 3/20
4695/4695 [==============================] - 0s 21us/step - loss: 0.4621 - acc: 0.7796
Epoch 4/20
4695/4695 [==============================]

4695/4695 [==============================] - 0s 51us/step - loss: 0.4218 - acc: 0.7989
Epoch 13/20
4695/4695 [==============================] - 0s 41us/step - loss: 0.4207 - acc: 0.7987
Epoch 14/20
4695/4695 [==============================] - 0s 44us/step - loss: 0.4199 - acc: 0.8006
Epoch 15/20
4695/4695 [==============================] - 0s 48us/step - loss: 0.4196 - acc: 0.8006
Epoch 16/20
4695/4695 [==============================] - 0s 38us/step - loss: 0.4190 - acc: 0.8004
Epoch 17/20
4695/4695 [==============================] - 0s 31us/step - loss: 0.4184 - acc: 0.8006
Epoch 18/20
4695/4695 [==============================] - 0s 34us/step - loss: 0.4182 - acc: 0.8000
Epoch 19/20
4695/4695 [==============================] - 0s 31us/step - loss: 0.4179 - acc: 0.8023
Epoch 20/20
4695/4695 [==============================] - 0s 17us/step
Epoch 1/20
4696/4696 [==============================] - 3s 721us/step - loss: 0.8602 - acc: 0.3301
Epoch 2/20
4696/4696 [=============================

4696/4696 [==============================] - 0s 48us/step - loss: 0.4138 - acc: 0.8041
Epoch 11/20
4696/4696 [==============================] - 0s 30us/step - loss: 0.4130 - acc: 0.8032
Epoch 12/20
4696/4696 [==============================] - 0s 23us/step - loss: 0.4135 - acc: 0.8013
Epoch 13/20
4696/4696 [==============================] - 0s 23us/step - loss: 0.4129 - acc: 0.8041
Epoch 14/20
4696/4696 [==============================] - 0s 30us/step - loss: 0.4134 - acc: 0.8022
Epoch 15/20
4696/4696 [==============================] - 0s 26us/step - loss: 0.4130 - acc: 0.8005
Epoch 16/20
4696/4696 [==============================] - 0s 27us/step - loss: 0.4129 - acc: 0.8007
Epoch 17/20
4696/4696 [==============================] - 0s 27us/step - loss: 0.4138 - acc: 0.8000
Epoch 18/20
4696/4696 [==============================] - 0s 27us/step - loss: 0.4127 - acc: 0.8009
Epoch 19/20
4696/4696 [==============================] - 0s 34us/step - loss: 0.4135 - acc: 0.8028
Epoch 20/20
4696/4696 

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?